# It's a Machine and Natural: Tagging

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_gum
from collections import defaultdict
from collections import Counter

seed = 691
all_docs = load_gum(num_articles = 0, seed = seed)
nsamp = min(25, len(all_docs)-1) ## this is the size of the test document base..., so nsamp=125 => test == 125 & train == 25!
test_docs = all_docs[:nsamp]
train_docs = all_docs[nsamp:]
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
covering_vocab = set([t for d in covering for s in d for t in s])

layers = [[[[row[2] for row in s] for s in d['conllu']] for d in train_docs],
          [[[row[3] for row in s] for s in d['conllu']] for d in train_docs]]
ltypes = ['lem','pos']

model = LM(form2ind={"<sos>": 0, "<pad>": 1}, covering_vocab = covering_vocab)
model.init(m = 10, zeta = 0.01, positional = True, seed = seed)
model.fit(docs, f'GUM-{nsamp}', covering = covering, layers = layers, ltypes = ltypes)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 135421.47it/s]


Training tokenizer...


Fitting:   7%|████████▋                                                                                                                   | 7/100 [00:15<03:19,  2.15s/it]


Built a vocabulary of 10984 types
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [03:43<00:00,  1.79s/it]


...

Absorbing pos-layer co-occurrences...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:15<00:00,  8.13it/s]


Computing marginal statistics...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5689877/5689877 [00:28<00:00, 202820.87it/s]


Done.
Model params, tokens, contexts, and % capacity used: 5689877 21922 442650 0.059


In [2]:
interpret_docs = list([docs[-2][:1]])
print(interpret_docs)
model.interpret(interpret_docs, seed = 691)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print('opening next sent:')
        for t in s._tokens:
            print('opening next token:')
            print((str(t), t._pos, t._lem, t._sep))

[['How to Prepare Quinoa ']]
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.74it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16710.37it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.88s/it]

opening next doc:
opening next sent:
opening next token:
('How', 'SCONJ', 'how', False)
opening next token:
(' ', 'PUNCT', ' ', False)
opening next token:
('to', 'PART', 'to', False)
opening next token:
(' ', 'PUNCT', ' ', False)
opening next token:
('Prepare', 'VERB', 'prepare', False)
opening next token:
(' ', 'PUNCT', ' ', False)
opening next token:
('Quinoa', 'DET', 'quinoa', False)
opening next token:
(' ', 'PUNCT', ' ', True)


In [3]:
interpret_docs = list([tdocs[0][1:2]])
print(interpret_docs)
model.interpret(interpret_docs, seed = 691)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print('opening next sent:')
        for t in s._tokens:
            print('opening next token:')
            print((str(t), t._pos, t._lem, t._sep))

[['Insights from Eye-Tracking ']]
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.16it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15768.06it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.77s/it]

opening next doc:
opening next sent:
opening next token:
('Insights', 'NOUN', 'degree', False)
opening next token:
(' ', 'PUNCT', ' ', False)
opening next token:
('from', 'ADP', 'from', False)
opening next token:
(' ', 'PUNCT', ' ', False)
opening next token:
('Eye', 'PROPN', 'the', False)
opening next token:
('-', 'PUNCT', ' ', False)
opening next token:
('Tracking', 'PROPN', 'year', False)
opening next token:
(' ', 'PUNCT', ' ', True)


In [4]:
interpret_docs = list([tdocs[0][1:2]])
print(interpret_docs)
interpret_covering =  [[[row[1] for row in s] for s in d['conllu']][1:2] for d in test_docs[0:1]]
print(interpret_covering)
model.interpret(interpret_docs, seed = 691, covering = interpret_covering)
interpret_gold =  [[[row[3] for row in s] for s in d['conllu']][1:2] for d in test_docs[0:1]]
print(interpret_covering)
accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
for doc_i, doc in enumerate(model._documents):
    print('opening next doc:')
    for sent_i, s in enumerate(doc._sentences):
        print('opening next sent:')
        for tok_i, t in enumerate(s._tokens):
            accuracy[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            accuracy_all.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            if str(t) != ' ':
                accuracy_nsp[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
                accuracy_all_nsp.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            print('opening next token:')
            print((str(t), t._pos, t._lem, t._sep, interpret_gold[doc_i][sent_i][tok_i]))
print("Tag-wise accuracy with/out space", {tag: sum(accuracy[tag])/len(accuracy[tag]) for tag in accuracy}, 
                                          {tag: sum(accuracy_nsp[tag])/len(accuracy_nsp[tag]) for tag in accuracy_nsp})
print("Overall accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))

[['Insights from Eye-Tracking ']]
[[['Insights', ' ', 'from', ' ', 'Eye-Tracking', ' ']]]
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.16it/s]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14665.40it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.44s/it]

[[['Insights', ' ', 'from', ' ', 'Eye-Tracking', ' ']]]
opening next doc:
opening next sent:
opening next token:
('Insights', 'NOUN', 'degree', False, 'NOUN')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('from', 'ADP', 'from', False, 'ADP')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Eye-Tracking', 'PUNCT', ' ', False, 'NOUN')
opening next token:
(' ', 'PUNCT', ' ', True, 'PUNCT')
Tag-wise accuracy with/out space {'NOUN': 0.5, 'PUNCT': 1.0, 'ADP': 1.0} {'NOUN': 0.5, 'ADP': 1.0}
Overall accuracy with/out space 0.8333333333333334 0.6666666666666666


In [5]:
interpret_docs = list([tdocs[4][:1]])
print(interpret_docs)
interpret_covering =  [[[row[1] for row in s] for s in d['conllu']][:1] for d in test_docs[4:5]]
model.interpret(interpret_docs, seed = 691, covering = interpret_covering)
interpret_gold =  [[[row[3] for row in s] for s in d['conllu']][:1] for d in test_docs[4:5]]
print(interpret_covering)
accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
for doc_i, doc in enumerate(model._documents):
    print('opening next doc:')
    for sent_i, s in enumerate(doc._sentences):
        print('opening next sent:')
        for tok_i, t in enumerate(s._tokens):
            accuracy[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            accuracy_all.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            if str(t) != ' ':
                accuracy_nsp[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
                accuracy_all_nsp.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            print('opening next token:')
            print((str(t), t._pos, t._lem, t._sep, interpret_gold[doc_i][sent_i][tok_i]))
print("Tag-wise accuracy with/out space", {tag: sum(accuracy[tag])/len(accuracy[tag]) for tag in accuracy}, 
                                          {tag: sum(accuracy_nsp[tag])/len(accuracy_nsp[tag]) for tag in accuracy_nsp})
print("Overall accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))

[['Distant Rhythm: Automatic Enjambment Detection on Four Centuries of Spanish Sonnets ']]
Tokenizing documents...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 28.18it/s]


Indexing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 9597.95it/s]


Interpreting documents


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.20s/it]

[[['Distant', ' ', 'Rhythm', ':', ' ', 'Automatic', ' ', 'Enjambment', ' ', 'Detection', ' ', 'on', ' ', 'Four', ' ', 'Centuries', ' ', 'of', ' ', 'Spanish', ' ', 'Sonnets', ' ']]]
opening next doc:
opening next sent:
opening next token:
('Distant', 'NOUN', 'mean', False, 'ADJ')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Rhythm', 'NOUN', ' ', False, 'NOUN')
opening next token:
(':', 'PUNCT', 'seem', False, 'PUNCT')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Automatic', 'NOUN', ' ', False, 'ADJ')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Enjambment', 'NOUN', ' ', False, 'NOUN')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Detection', 'NOUN', ' ', False, 'NOUN')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('on', 'ADP', 'on', False, 'ADP')
opening next token:
(' ', 'PUNCT', ' ', False, 'PUNCT')
opening next token:
('Four

In [6]:
import numpy as np
print("model vector dimension: ", model._documents[0]._sentences[0]._tokens[0]._whatevers[0]._vec.shape[0])
model._documents[0]._sentences[0]._tokens[0]._whatevers[0]._vec.shape[0]
nspv = model._documents[0]._sentences[0]._tokens[0]._whatevers[-1]._vec; nspvn = np.linalg.norm(nspv)
spv1 = model._documents[0]._sentences[0]._tokens[1]._whatevers[-1]._vec; spv1n = np.linalg.norm(spv1)
spv2 = model._documents[0]._sentences[0]._tokens[4]._whatevers[-1]._vec; spv2n = np.linalg.norm(spv2)

print("vector similarity of non-space to space whatevers (first two), \nvs. similarity of separate space tokens (last one): \n\n",
      nspv.dot(spv1)/(nspvn*spv1n), nspv.dot(spv2)/(nspvn*spv2n), spv1.dot(spv2)/(spv1n*spv2n))

model vector dimension:  10984
vector similarity of non-space to space whatevers (first two), 
vs. similarity of separate space tokens (last one): 

 0.996383687675028 0.9945622841576942 0.9997247105385825


In [7]:
accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
model.interpret(tdocs, seed = 691, covering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs], 
                ltypes = ['lem'],  layers = [[[[row[2] for row in s] for s in d['conllu']] for d in test_docs]], vecs = False)
interpret_gold =  [[[row[3] for row in s] for s in d['conllu']] for d in test_docs]
for doc_i, doc in enumerate(model._documents):
    for sent_i, s in enumerate(doc._sentences):
        for tok_i, t in enumerate(s._tokens):
            accuracy[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            accuracy_all.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
            if str(t) != ' ':
                accuracy_nsp[interpret_gold[doc_i][sent_i][tok_i]].append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)
                accuracy_all_nsp.append(interpret_gold[doc_i][sent_i][tok_i]==t._pos)

print("Overall accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))

Tokenizing documents...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:51<00:00,  2.07s/it]


Indexing documents...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 541.87it/s]


Processing lem-tags for whatever-layer prediction...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 162.88it/s]


Interpreting documents


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:53<00:00,  2.14s/it]

Overall accuracy with/out space 0.8559518186112424 0.73962860529435


In [8]:
"Tag-wise accuracy with space", list(Counter({tag: (sum(accuracy[tag])/len(accuracy[tag]), len(accuracy[tag])) for tag in accuracy}).most_common())

('Tag-wise accuracy with space',
 [('PUNCT', (0.9667164310178273, 22774)),
  ('ADP', (0.9113409678611009, 2707)),
  ('NOUN', (0.8836094158674804, 4588)),
  ('DET', (0.8683417085427135, 1990)),
  ('CCONJ', (0.7891061452513967, 716)),
  ('AUX', (0.7050632911392405, 790)),
  ('VERB', (0.674937965260546, 2015)),
  ('PRON', (0.6707589285714286, 896)),
  ('NUM', (0.6593707250341997, 731)),
  ('ADJ', (0.6036550542547116, 1751)),
  ('PART', (0.5606796116504854, 412)),
  ('ADV', (0.4393939393939394, 726)),
  ('PROPN', (0.37850467289719625, 1712)),
  ('SCONJ', (0.3643410852713178, 387)),
  ('SYM', (0.18032786885245902, 61)),
  ('INTJ', (0.09090909090909091, 11)),
  ('X', (0.0410958904109589, 73))])

In [9]:
"Tag-wise accuracy without space", list(Counter({tag: (sum(accuracy_nsp[tag])/len(accuracy_nsp[tag]), len(accuracy_nsp[tag])) for tag in accuracy_nsp}).most_common())

('Tag-wise accuracy without space',
 [('ADP', (0.921209858103062, 2678)),
  ('NOUN', (0.8898156277436348, 4556)),
  ('DET', (0.874051593323217, 1977)),
  ('CCONJ', (0.7980225988700564, 708)),
  ('PUNCT', (0.7788004750593824, 3368)),
  ('AUX', (0.7077509529860229, 787)),
  ('VERB', (0.6783042394014963, 2005)),
  ('PRON', (0.6745230078563412, 891)),
  ('NUM', (0.6666666666666666, 723)),
  ('ADJ', (0.6088709677419355, 1736)),
  ('PART', (0.5661764705882353, 408)),
  ('ADV', (0.44367176634214184, 719)),
  ('PROPN', (0.38095238095238093, 1701)),
  ('SCONJ', (0.36910994764397903, 382)),
  ('SYM', (0.1896551724137931, 58)),
  ('INTJ', (0.09090909090909091, 11)),
  ('X', (0.04225352112676056, 71))])